# Custom Metrics Server 

### Train and serialise logistic regressor

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

dataset = load_iris()
feature_names = dataset.feature_names
class_names = list(dataset.target_names)

X = dataset.data
y = dataset.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LogisticRegression(max_iter=4000)
lr.fit(X_train, y_train)

# Saving the logistic regressor
joblib.dump(lr, 'model.joblib')

Push to GCP

In [ ]:
YOUR_NAME = "john-smith"
!gsutil cp model.joblib gs://tom-seldon-examples/deploy-workshop/"{YOUR_NAME}"/lr/model.joblib

### Custom Metrics Server 

First we need to specify custom metric logic within a class. We can adapt logic from [existing custom metric servers](https://github.com/SeldonIO/seldon-core/tree/22d19a1b7d445b41d9588ab2e525a64ef5c10097/components/alibi-detect-server/adserver/cm_models) in Seldon Core.

In [ ]:
class MultiClassNumeric:
    """
    MultiClassNumeric Model
    Parameters
    -----------
    """

    def __init__(self):
        pass

    def transform(self, truth, response, request=None):
        """
        Perform a multiclass numeric comparison between truth and response.
        Parameters
        -----------
        truth
            Actual data value as format of <number> or [<number>]
        response
            Prediction data value as format of <number> or [<number>]
        request
            Input data value as format of <number> or [<number>]
        """

        metrics = []

        response_class = (
            response[0] if isinstance(response, (list, np.ndarray)) else response
        )
        truth_class = truth[0] if isinstance(truth, (list, np.ndarray)) else truth

        correct = response_class == truth_class

        if correct:
            metrics.append(
                {
                    "key": "seldon_metric_true_positive",
                    "type": "COUNTER",
                    "value": 1,
                    "tags": {"class": f"CLASS_{truth_class}"},
                }
            )
            metrics.append(
                {
                    "key": "seldon_metric_true_negative",
                    "type": "COUNTER",
                    "value": 1,
                    "tags": {"class": f"CLASS_{response_class}"},
                }
            )
        else:
            metrics.append(
                {
                    "key": "seldon_metric_false_negative",
                    "type": "COUNTER",
                    "value": 1,
                    "tags": {"class": f"CLASS_{truth_class}"},
                }
            )
            metrics.append(
                {
                    "key": "seldon_metric_false_positive",
                    "type": "COUNTER",
                    "value": 1,
                    "tags": {"class": f"CLASS_{response_class}"},
                }
            )

        return SeldonResponse(None, None, metrics)

Serialise class:

In [ ]:
import pickle 
with open('meta.pickle', 'wb') as f:
    pickle.dump(MultiClassNumeric(), f)

Push to s3 bucket: 

In [ ]:
!gsutil cp meta.pickle gs://tom-seldon-examples/deploy-workshop/"{YOUR_NAME}"/metrics/meta.pickle

In [ ]:
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, MetricsServerApi
from seldon_deploy_sdk.auth import OIDCAuthenticator
import requests

SD_IP = "34.90.29.195"

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"

config.oidc_client_id = "sd-api"
config.oidc_client_secret = "sd-api-secret"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"

def auth():
    auth = OIDCAuthenticator(config)
    config.access_token = auth.authenticate("admin@seldon.io", "12341234")
    api_client = ApiClient(config)
    return api_client

In [ ]:
MODEL_NAME = "lr"

DEPLOYMENT_NAME = f"{YOUR_NAME}-{MODEL_NAME}"
MODEL_LOCATION = f"gs://tom-seldon-examples/deploy-workshop/{YOUR_NAME}/{MODEL_NAME}"

In [ ]:
NAMESPACE = "test"
PREPACKAGED_SERVER = "SKLEARN_SERVER"

CPU_REQUESTS = "1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "1"
MEMORY_LIMITS = "1Gi"

mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "transport": "rest",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "implementation": PREPACKAGED_SERVER,
                    "modelUri": MODEL_LOCATION,
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "endpoint": {
                        "type": "REST"
                    },
                    "parameters": [],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

In [ ]:
metrics_server = {
    "params": {
        "model_name": "multiclassserver",
        "protocol": "seldonfeedback.http",
        "reply_url": "http://seldon-request-logger.seldon-logs",
        "storage_uri": f"gs://tom-seldon-examples/deploy-workshop/{YOUR_NAME}/metrics"  
    }
}

In [ ]:
metrics_api = MetricsServerApi(auth())
metrics_api.create_metrics_server_seldon_deployment(name= DEPLOYMENT_NAME, namespace= NAMESPACE, metrics_server=metrics_server)
